In [1]:
!pip install -U git+https://github.com/huggingface/diffusers.git
!pip install -U controlnet_aux==0.0.7 
!pip install transformers accelerate safetensors

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/huggingface/diffusers.git to c:\users\platinum\appdata\local\temp\pip-req-build-10h9ya16
  Resolved https://github.com/huggingface/diffusers.git to commit f912f39b50f87e50a9d99346f5c1b6e644653262
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git 'C:\Users\Platinum\AppData\Local\Temp\pip-req-build-10h9ya16'


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [1]:
from diffusers import StableDiffusionXLAdapterPipeline, T2IAdapter, EulerAncestralDiscreteScheduler, AutoencoderKL
from diffusers.utils import load_image, make_image_grid
from controlnet_aux.pidi import PidiNetDetector
import torch

C:\Users\Platinum\anaconda3\envs\cuda_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# load adapter
adapter = T2IAdapter.from_pretrained(
  "TencentARC/t2i-adapter-sketch-sdxl-1.0", torch_dtype=torch.float, varient="fp16"
).to("cpu")

In [8]:

# load euler_a scheduler
model_id = 'stabilityai/stable-diffusion-xl-base-1.0'
euler_a = EulerAncestralDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
vae=AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float)
pipe = StableDiffusionXLAdapterPipeline.from_pretrained(
    model_id, vae=vae, adapter=adapter, scheduler=euler_a, torch_dtype=torch.float, variant="fp16", 
).to("cpu")
#pipe.enable_xformers_memory_efficient_attention()

pidinet = PidiNetDetector.from_pretrained("lllyasviel/Annotators").to("cpu")

Loading pipeline components...: 100%|██████████| 7/7 [00:33<00:00,  4.82s/it]


In [9]:
url = "https://huggingface.co/Adapter/t2iadapter/resolve/main/figs_SDXLV1.0/org_sketch.png"
image = load_image(url)
image = pidinet(
  image, detect_resolution=1024, image_resolution=1024, apply_filter=True
)

In [10]:
prompt = "a robot, mount fuji in the background, 4k photo, highly detailed"
negative_prompt = "extra digit, fewer digits, cropped, worst quality, low quality, glitch, deformed, mutated, ugly, disfigured"

gen_images = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    image=image,
    num_inference_steps=30,
    adapter_conditioning_scale=0.9,
    guidance_scale=7.5, 
).images[0]
gen_images.save('out_sketch.png')

100%|██████████| 30/30 [48:42<00:00, 97.41s/it] 


In [11]:
image2 = load_image("tree.jpg")
image2 = pidinet(
  image2, detect_resolution=1024, image_resolution=1024, apply_filter=True
)

In [13]:
prompt = "a tree, mount in the background, 4k photo, highly detailed"
negative_prompt = ""

gen_images = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    image=image2,
    num_inference_steps=10,
    adapter_conditioning_scale=0.9,
    guidance_scale=7.5, 
).images[0]
gen_images.save('out_tree.png')

100%|██████████| 10/10 [14:51<00:00, 89.19s/it]


In [17]:
image2 = load_image("manga.jpeg")
image2 = pidinet(
  image2, detect_resolution=1024, image_resolution=1024, apply_filter=True
)
prompt = "a manga, purple jacket, blonde, green eyes, 4k photo, highly detailed"
negative_prompt = ""

gen_images = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    image=image2,
    num_inference_steps=10,
    adapter_conditioning_scale=0.9,
    guidance_scale=7.5, 
).images[0]
gen_images.save('out_manga.png')

100%|██████████| 10/10 [12:41<00:00, 76.14s/it]
